In [1]:
import numpy as np
import pandas as pd
import numba as nb
import matplotlib.pyplot as plt
import time
from pathlib import Path
from IDT_alg_VR_centred_SMI import IDTVR

In [8]:
from numpy import float64


df_et = pd.read_csv(
    "./data/VR/Logfiles_All/GazeData/p001_gazelog_on_machine_GFK-VR_date2017.6.14_time8.49.52.csv",
    skiprows=15,
    sep=";",
    encoding="utf-8",
)

# head pose
df_et[["head_pos_x", "head_pos_y", "head_pos_z"]] = df_et[
    "head_pos"
].str.split(",", expand=True)

# gaze vector
df_et[["et_x", "et_y", "et_z"]] = df_et["bino_hitObject_pos"].str.split(
    ",", expand=True
)

# POR
df_et[["POR X", "POR Y"]] = df_et["smi_bino_por"].str.split(
    ",", expand=True
)

# fix timestamp
firstTimestamp = df_et.loc[0, "#timestamp_unity"]
if isinstance(firstTimestamp, float):
    df_et["elapsedTime"] = df_et["#timestamp_unity"] - firstTimestamp
else:
    raise(TypeError("Wrong firstTimestamp"))

print("Shape", df_et.shape)
print("Columns", df_et.columns)
df_et.head(8)

Shape (45883, 45)
Columns Index(['#timestamp_unity', 'timestamp_smi', 'head_pos', 'head_dir',
       'smi_bino_por', 'smi_left_por', 'smi_left_base', 'smi_left_dir',
       'smi_right_por', 'smi_right_base', 'smi_right_dir',
       'bino_gazePos_global', 'bino_hitObject', 'bino_gazePos_local',
       'bino_hitObject_pos', 'bino_hitObject_dir', 'bino_hitObject_scl',
       'bino_hitObject_TexCoord', 'bino_hitObject_Feature',
       'left_gazePos_global', 'left_hitObject', 'left_gazePos_local',
       'left_hitObject_pos', 'left_hitObject_dir', 'left_hitObject_scl',
       'left_hitObject_TexCoord', 'left_hitObject_Feature',
       'right_gazePos_global', 'right_hitObject', 'right_gazePos_local',
       'right_hitObject_pos', 'right_hitObject_dir', 'right_hitObject_scl',
       'right_hitObject_TexCoord', 'right_hitObject_Feature', 'Unnamed: 35',
       'head_pos_x', 'head_pos_y', 'head_pos_z', 'et_x', 'et_y', 'et_z',
       'POR X', 'POR Y', 'elapsedTime'],
      dtype='object')


,#timestamp_unity,timestamp_smi,head_pos,head_dir,smi_bino_por,smi_left_por,smi_left_base,smi_left_dir,smi_right_por,smi_right_base,...,Unnamed: 35,head_pos_x,head_pos_y,head_pos_z,et_x,et_y,et_z,POR X,POR Y,elapsedTime
0,1186.836,1.234381e+12,"-1.31369,1.77953,-0.6532125","2.24284,79.3271,5.289413","1061.494,646.725","1039.435,649.41","-0.03316501,0.0005038494,-0.03630815","-0.01179717,-0.1173718,0.993018","1083.553,644.0402","0.02889526,-0.001987986,-0.03259998",...,NaN,-1.31369,1.77953,-0.6532125,0.3730002,-0.9300001,-0.09417015,1061.494,646.725,0.000
1,1187.170,1.243201e+12,"-1.289178,1.767716,-0.6821822","10.17717,103.0916,358.2414","1101.536,811.8827","1072.272,806.0854","-0.03301297,0.001528281,-0.03567222","0.007515335,-0.4412687,0.8973435","1130.799,817.6799","0.02854932,-0.0007540563,-0.03147267",...,NaN,-1.289178,1.767716,-0.6821822,0.3730002,-0.9300001,-0.09417015,1101.536,811.8827,0.334
2,1187.503,1.243209e+12,"-1.340189,1.761135,-0.6955911","12.74784,137.9351,0.5791597","1099.333,812.0182","1070.997,805.8777","-0.03304359,0.001476976,-0.0356314","0.006419565,-0.4463517,0.8948346","1127.669,818.1587","0.02854932,-0.0007540563,-0.03147267",...,NaN,-1.340189,1.761135,-0.6955911,0.02799988,0.7062,-1.062701,1099.333,812.0182,0.667
3,1187.511,1.243218e+12,"-1.33947,1.760692,-0.6950987","12.85189,137.479,0.3577516","1097.746,812.194","1070.126,805.8381","-0.03305491,0.001403628,-0.03551621","0.008930573,-0.4458749,0.8950508","1125.366,818.5497","0.02854932,-0.0007540563,-0.03147267",...,NaN,-1.33947,1.760692,-0.6950987,-0.4064397,0.6755455,-1.539988,1097.746,812.194,0.675
4,1187.522,1.243242e+12,"-1.337413,1.759235,-0.6935291","13.40208,136.1206,359.791","1099.942,811.7125","1073.047,807.2813","-0.03313671,0.00128562,-0.03564808","0.01993912,-0.4464585,0.8945822","1126.837,816.1439","0.0284521,-0.001061178,-0.03142802",...,NaN,-1.337413,1.759235,-0.6935291,-0.4064397,0.6755455,-1.539988,1099.942,811.7125,0.686
5,1187.545,1.243265e+12,"-1.335713,1.757644,-0.6921561","14.15824,135.0944,359.6538","1037.824,810.369","1016.023,802.5681","-0.03300181,0.001087183,-0.03563282","-0.06141639,-0.4217822,0.9046147","1059.624,818.1699","0.02859335,-0.0008502628,-0.03071016",...,NaN,-1.335713,1.757644,-0.6921561,-0.4064397,0.6755455,-1.539988,1037.824,810.369,0.709
6,1187.567,1.243273e+12,"-1.335517,1.757422,-0.6921113","14.28801,135.0624,359.7637","1024.759,805.1152","997.149,798.1165","-0.03296184,0.001041947,-0.03553583","-0.08337206,-0.4126913,0.9070474","1052.369,812.114","0.0285535,-0.0009895682,-0.03074798",...,NaN,-1.335517,1.757422,-0.6921113,-0.4064397,0.6755455,-1.539988,1024.759,805.1152,0.731
7,1187.578,1.243289e+12,"-1.334649,1.756655,-0.6913705","14.66247,134.6094,359.7735","1019.526,802.3407","996.0416,791.1038","-0.03294502,0.00119712,-0.03608138","-0.07116397,-0.4120487,0.9083785","1043.011,813.5777","0.02857602,-0.0008649386,-0.02931023",...,NaN,-1.334649,1.756655,-0.6913705,-0.4064397,0.6755455,-1.539988,1019.526,802.3407,0.742


In [ ]:
df_et = df_et[
    [
        "head_pos_x",
        "head_pos_y",
        "head_pos_z",
        "et_x",
        "et_y",
        "et_z",
        "POR X",
        "POR Y",
        "elapsedTime",
        "bino_hitObject",
        "bino_hitObject_Feature",
    ]
]

# cast columns to float (except the last 3)
df_et[df_et.columns[:-3]] = df_et[df_et.columns[:-3]].astype(float)

# calculate time for seach sample
df_et["time_delta"] = df_et["elapsedTime"] - df_et["elapsedTime"].shift(1)

print("Shape", df_et.shape)
print("Columns", df_et.columns)

df_et.head(8)

In [ ]:
idt_vr = IDTVR(numba_allow=True, time_th=0.1)

t0 = time.time()
df_et_res, df_et_fixations = idt_vr.fit_compute(df_et, time="elapsedTime", debug=True)
delta_time_numba = time.time() - t0

df_et_fixations

In [ ]:
print(
    "Percentage of fixations detected",
    np.round(100 * np.mean(df_et_res["class_disp"] == 0), 2),
    "%",
)
print(
    "Percentage of saccades detected",
    np.round(100 * np.mean(df_et_res["class_disp"] == 1), 2),
    "%",
)
print("Delta time", np.round(delta_time_numba, 4), "s")


In [ ]:
df_et_fixations['duration'].describe()

In [ ]:
ivt_file = Path("C://Users/tbswe/data/et-data/VR/Logfiles_All/GazeData/p001_fixations.csv")

#fix enconding
fi = ivt_file.open('rb')
content = fi.read()
fi.close()
ivt_file_fixed = Path("C://Users/tbswe/data/et-data/VR/Logfiles_All/GazeData/p001_fixations_fixed.csv")
fo = ivt_file_fixed.open('wb')
fo.write(content.replace(b'\x00', b''))
fo.close()

df_ivt = pd.read_csv(
    ivt_file_fixed,
    sep=",",
    encoding="iso-8859-1",
)

In [ ]:
df_ivt['gazeevent_duration'].describe()